In [2]:
import numpy as np
import pandas as pd

In [ ]:
data_path = './data/'
data_id = pd.read_csv(data_path + 'data_identification.csv')
emotions = pd.read_csv(data_path + 'emotion.csv')
sample_sub = pd.read_csv(data_path + 'sampleSubmission.csv')
tweets = pd.read_json(data_path + 'tweets_DM.json', lines=True)
tweets_important = pd.DataFrame(tweets._source)


In [ ]:
tw_list = tweets_important['_source'].to_list()
tmp_df = pd.DataFrame.from_records(tw_list) #dict list to dataframe
tmp_df_list = tmp_df['tweet'].to_list()
final_tweet_df = pd.DataFrame.from_records(tmp_df_list)
df_final = pd.merge(final_tweet_df, data_id, how='outer', on='tweet_id').merge(emotions, how='outer', on='tweet_id')

In [ ]:
final_tweet_df

In [5]:
df_final

,hashtags,tweet_id,text,identification,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,NaN
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,NaN
...,...,...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test,NaN
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test,NaN
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,test,NaN
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",train,joy


In [6]:
emotion2idx = {}
idx2emotion = {}
for i, emotion in enumerate(df_final['emotion'].unique()):
    emotion2idx[emotion] = i
    idx2emotion[i] = emotion

print(emotion2idx)
print(idx2emotion)


{'anticipation': 0, 'sadness': 1, nan: 2, 'fear': 3, 'joy': 4, 'anger': 5, 'trust': 6, 'disgust': 7, 'surprise': 8}
{0: 'anticipation', 1: 'sadness', 2: nan, 3: 'fear', 4: 'joy', 5: 'anger', 6: 'trust', 7: 'disgust', 8: 'surprise'}


In [ ]:
import preprocessor as p 
def preprocess_tweet(text):
    text_ = []
    for t in text.split():
        if t == '<LH>':
            continue
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        t = t.lower()        
        text_.append(t)
    return " ".join(text_)


In [8]:
#text = df_final.iloc[10].text
#print(text)
#print(preprocess_tweet(text))

In [9]:
df_final['text'] = df_final['text'].apply(preprocess_tweet)

In [10]:
train_df = df_final[df_final['identification'] == 'train']
test_df = df_final[df_final['identification'] == 'test']


In [11]:
train_df = train_df.replace('NaN', np.nan)
train_df = train_df.dropna(subset=['emotion'])


In [ ]:
test_df.to_csv('test.csv', index=False)
train_df.to_csv('train.csv', index=False)
